In [1]:
import pandas as pd

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [10]:
import datasets
from datasets import Dataset, DatasetDict, load_dataset

In [3]:
dataset = load_dataset("lavita/medical-qa-datasets", "chatdoctor_healthcaremagic", split = "train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/16.4k [00:00<?, ?B/s]

(…)-00000-of-00001-64869eb716edea01.parquet:   0%|          | 0.00/70.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

In [4]:
df = dataset.to_pandas()

In [5]:
df

,instruction,input,output
0,"If you are a doctor, please answer the medical...",I woke up this morning feeling the whole room ...,"Hi, Thank you for posting your query. The most..."
1,"If you are a doctor, please answer the medical...",My baby has been pooing 5-6 times a day for a ...,Hi... Thank you for consulting in Chat Doctor....
2,"If you are a doctor, please answer the medical...","Hello, My husband is taking Oxycodone due to a...","Hello, and I hope I can help you today.First, ..."
3,"If you are a doctor, please answer the medical...",lump under left nipple and stomach pain (male)...,HI. You have two different problems. The lump ...
4,"If you are a doctor, please answer the medical...",I have a 5 month old baby who is very congeste...,Thank you for using Chat Doctor. I would sugge...
...,...,...,...
112160,"If you are a doctor, please answer the medical...",im 25 years old ..i started using mtp kit on 5...,"Hello, Thanks for letting us know your health ..."
112161,"If you are a doctor, please answer the medical...",My 5 year old son has been coughing for a mont...,As you have mentioned in your history that you...
112162,"If you are a doctor, please answer the medical...",My toes on right foot more than left are numb ...,Hi. The numbness and blue discoloration could ...
112163,"If you are a doctor, please answer the medical...","I was diagnosis with pleurisy last Tuesday, an...",Thanks for your question on Chat Doctor. Treat...


In [6]:
def llama_prompt(row):
  return f"""<|start_header_id|>system<|end_header_id|>{row["instruction"]}<|eot_id|><|start_header_id|>user<|end_header_id|> This is the question: {row["input"]}<|eot_id|><|start_header_id|>assistant<|end_header_id|>{row["output"]}<|eot_id|>"""

In [7]:
df = pd.DataFrame(df)

In [8]:
df['prompt'] = df.apply(llama_prompt, axis=1)

In [9]:
df.head()

,instruction,input,output,prompt
0,"If you are a doctor, please answer the medical...",I woke up this morning feeling the whole room ...,"Hi, Thank you for posting your query. The most...",<|start_header_id|>system<|end_header_id|>If y...
1,"If you are a doctor, please answer the medical...",My baby has been pooing 5-6 times a day for a ...,Hi... Thank you for consulting in Chat Doctor....,<|start_header_id|>system<|end_header_id|>If y...
2,"If you are a doctor, please answer the medical...","Hello, My husband is taking Oxycodone due to a...","Hello, and I hope I can help you today.First, ...",<|start_header_id|>system<|end_header_id|>If y...
3,"If you are a doctor, please answer the medical...",lump under left nipple and stomach pain (male)...,HI. You have two different problems. The lump ...,<|start_header_id|>system<|end_header_id|>If y...
4,"If you are a doctor, please answer the medical...",I have a 5 month old baby who is very congeste...,Thank you for using Chat Doctor. I would sugge...,<|start_header_id|>system<|end_header_id|>If y...


In [11]:
dataset = datasets.Dataset.from_pandas(df)

In [12]:
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)

dataset_dict = DatasetDict({
    "train": train_test_split["train"],
    "test": train_test_split["test"]
})

In [13]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt'],
        num_rows: 89732
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'prompt'],
        num_rows: 22433
    })
})

In [14]:
dataset_dict.push_to_hub("sathvik123/llama3-medical-dataset",token=" ")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/90 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/23 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/sathvik123/llama3-medical-dataset/commit/3828da0ab93a9008b4dcc58878c5c5728f2203a9', commit_message='Upload dataset', commit_description='', oid='3828da0ab93a9008b4dcc58878c5c5728f2203a9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/sathvik123/llama3-medical-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='sathvik123/llama3-medical-dataset'), pr_revision=None, pr_num=None)